In [1]:
import string
import nltk
from nltk import word_tokenize, FreqDist
from nltk.corpus import stopwords
import pandas as pd

In [2]:
pf = pd.read_csv('job_details_all_without_duplicate.csv')

In [3]:
pf = pf[["job_title", "job_desc"]]

In [4]:
pf.head()

,job_title,job_desc
0,(Urgent!) IT Support,Permanent\nLogging and processing support call...
1,.Net Developer,Job Summary\nAbout Us\nInfosys is a leading pr...
2,.Net Software Engineers,Joining Accenture provides you with the opport...
3,2nd / 3rd Line Technical Support Engineer,Permanent\nRoles & Responsibilities\nOn a day-...
4,AMOS Support Specialist,Roles & Responsibilities\nLufthansa Systems is...


In [5]:
punctuation = string.punctuation
punctuation = punctuation + '-' + '–'

conditions = [
#     lambda t: len(t) < 3,
    lambda t: t in punctuation,
    lambda t: any(c.isdigit() for c in t),
]

stop = stopwords.words('english')
custom_stop = ['job', 'summary', 'description', 'work', 'working', 'type', 'time', 'team', 'full', 'singapore', 'location', 'requirements', '’', 'about', 'us',
               'experience', 'skills', 'world', 'build', "'s"]
stop.extend(custom_stop)

porter = nltk.PorterStemmer()

# Pre_process text, remove words is punctuation or digit, remove stop words, use stemmer to get general text
def pre_process(desc, conditions, stop, porter):
    tokens_nop = [ t for t in word_tokenize(desc) if not any(f(t) for f in conditions)]
    tokens_lower = [ t.lower() for t in tokens_nop ]
    tokens_nostop = [ t for t in tokens_lower if t not in stop ]
#     tokens_nostop = set(tokens_nostop) # get the unique words in tokens 
    tokens_porter = [ porter.stem(t) for t in tokens_nostop ] # get the stemmed words
    return tokens_nostop

In [6]:
# Pre_process job_title + job_desc, add new column tokens
pf["tokens"] = pf.apply(lambda row:pre_process(row.job_title + " " + row.job_desc, conditions, stop, porter), axis=1)
pf.tokens = pf.tokens.str.join(" ")

# Pre_process job_desc
# pf["tokens"] = pf.job_desc.apply(lambda x: pre_process(x, conditions, stop))
# Pre_process 1 of job_desc
# pre_process(pf.job_desc.iloc[1], conditions, stop)

In [16]:
pf.head()

,job_title,job_desc,tokens
0,(Urgent!) IT Support,Permanent\nLogging and processing support call...,urgent support permanent logging processing su...
1,.Net Developer,Job Summary\nAbout Us\nInfosys is a leading pr...,.net developer infosys leading provider next-g...
2,.Net Software Engineers,Joining Accenture provides you with the opport...,.net software engineers joining accenture prov...
3,2nd / 3rd Line Technical Support Engineer,Permanent\nRoles & Responsibilities\nOn a day-...,line technical support engineer permanent role...
4,AMOS Support Specialist,Roles & Responsibilities\nLufthansa Systems is...,amos support specialist roles responsibilities...


In [17]:
pf.to_csv('job_details_processed.csv', index=False)